In [1]:
# useful to autoreload the module without restarting the kernel
%load_ext autoreload
%autoreload 2

In [2]:
from mppi import InputFiles as I, Calculators as C, Datasets as D, Parsers as P, Utilities as U
from mppi.Utilities import Tools
from mppi.Utilities import Constants as Const
import matplotlib.pyplot as plt
import numpy as np
import os, yaml

In [3]:
omp = 1
mpi = 16 #4
sbatch_options = ['--partition=all12h']

In [4]:
#code = C.YamboCalculator(omp=omp,mpi=mpi,executable='yambo')
code = C.YamboCalculator(omp=omp,mpi=mpi,executable='yambo',scheduler='slurm',sbatch_options=sbatch_options)
code.global_options()

Initialize a Yambo calculator with scheduler slurm


{'omp': 1,
 'mpi': 16,
 'mpi_run': 'mpirun -np',
 'executable': 'yambo',
 'scheduler': 'slurm',
 'skip': True,
 'clean_restart': True,
 'dry_run': False,
 'wait_end_run': True,
 'sbatch_options': ['--partition=all12h'],
 'activate_BeeOND': True,
 'verbose': True}

# GW convergence analysis of the Germanene

We compute the GW corrections of the DFT band structure of the Germanene. 

We discuss the convergene procedure using various approximations of the self-energy.

We have performed various preliminary tests in particular:

__Parallelization strategy__

We set the SE_ROLEs as 'b.qp.q' and we observe that
* for 6,6,1 kpoints the highest performance are achieved with 'mpi.1.1'
* for 12,12,1 kpoints the highest performance are achieved with ....

## Hartree-Fock convergence analysis 

At the Hartree-Fock (HF) level the self energy is limited to the exchange part $\Sigma_x$ written in term of the
bare Coulomb potential.

The diagonal matrix elements of $\Sigma_x$ on the KS states are written as
$$
\langle nk|\Sigma_x|nk \rangle = -\sum_{n'}\int_{BZ}dq\sum_G \frac{4\pi}{|q+G|^2}
|\rho_{nn'}(k,q,G|^2f_{n',k-q}
$$
The sum over $n'$ il limited to the full bands due the $f_{n'}$ factor and so the self energy in this
approximation scheme depends on the $k$-points sampling in the BZ (q sum) and on the cutoff of the plane wave
expansio of the KS states.

In [5]:
source_dir = 'DFT_analysis/kpoints_12-nbnds_60.save'
run_dir = 'GW_HF_analysis/kpoints_12'

#source_dir = 'DFT_analysis/kpoints_6-nbnds_40.save'
#run_dir = 'GW_HF_analysis/kpoints_6'

#run_dir = 'Test_GaAs'

In [6]:
Tools.build_SAVE(source_dir,run_dir,command='p2y')

Create folder GW_HF_analysis/kpoints_12
Executing command: cd DFT_analysis/kpoints_12-nbnds_60.save; p2y
Create a symlink of /work/dalessandro/2D_ELECTRONIC_OPTICAL_PROPERTIES/GERMANENE/DFT_analysis/kpoints_12-nbnds_60.save/SAVE in GW_HF_analysis/kpoints_12
Executing command: cd GW_HF_analysis/kpoints_12;OMP_NUM_THREADS=1 yambo


We build the input for HF analysis including the random integration method (RIM) for the small $q$ component of the Coulomb potential
and the cutoff of the Coulomb potential in the off plane direction

In [11]:
inp = I.YamboInput(args='yambo -r -x -V all',folder=run_dir)
inp.set_extendOut()
inp.set_kRange(19,19)
inp.set_bandRange(27,30)
inp['variables']['RandQpts'] = [1000000,'']
inp['variables']['RandGvec'] = [100,'']
inp['variables']['CUTGeo'] = 'box z'
inp['variables']['CUTBox'] = [[0.,0.,32],'au']
inp['variables']['EXXRLvcs'] = [1.0,'Ha']

inp['variables']['SE_CPU'] = '8.1.2'
inp['variables']['SE_ROLEs'] = 'b.qp.q'
inp

{'args': 'yambo -r -x -V all',
 'folder': 'GW_HF_analysis/kpoints_12',
 'filename': 'yambo.in',
 'arguments': ['rim_cut', 'HF_and_locXC', 'ExtendOut'],
 'variables': {'StdoHash': [40.0, ''],
  'Nelectro': [28.0, ''],
  'ElecTemp': [0.0, 'eV'],
  'BoseTemp': [-1.0, 'eV'],
  'OccTresh': [1e-05, ''],
  'NLogCPUs': [0.0, ''],
  'MEM_tresh': [186671.0, 'Kb'],
  'FFTGvecs': [52407.0, 'RL'],
  'SE_Threads': [0.0, ''],
  'RandQpts': [1000000, ''],
  'RandGvec': [100, ''],
  'IDEm1Ref': [0.0, ''],
  'CUTRadius': [0.0, ''],
  'CUTCylLen': [0.0, ''],
  'CUTwsGvec': [0.7, ''],
  'EXXRLvcs': [1.0, 'Ha'],
  'VXCRLvcs': [356277.0, 'RL'],
  'DBsIOoff': 'none',
  'DBsFRAGpm': 'none',
  'PAR_def_mode': 'balanced',
  'CUTGeo': 'box z',
  'Em1Anys': [[0.0, 0.0, 0.0], ''],
  'CUTBox': [[0.0, 0.0, 32], 'au'],
  'QPkrange': [[19, 19, 27, 30], ''],
  'QPerange': [[1, 19, 0.0, '-1.000000'], ''],
  'SE_CPU': '8.1.2',
  'SE_ROLEs': 'b.qp.q'}}

In [12]:
name = 'hf_2'

In [ ]:
results = code.run(run_dir=run_dir,input=inp,name=name,skip=False)
results

run command: mpirun -np 16 yambo -F hf_2.in -J hf_2 -C hf_2
slurm submit:  cd GW_HF_analysis/kpoints_12 ; sbatch job_hf_2.sh
computation hf_2 is running...


In [ ]:
###########################################################

In [52]:
dft = P.YamboDftParser(os.path.join(run_dir,'SAVE','ns.db1'))

Parse file : GW_HF_analysis/kpoints_12/SAVE/ns.db1


In [53]:
dft.kpoints

array([[ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.08333333,  0.        ],
       [ 0.        ,  0.16666667,  0.        ],
       [ 0.        ,  0.25      ,  0.        ],
       [ 0.        ,  0.33333333,  0.        ],
       [ 0.        ,  0.41666667,  0.        ],
       [ 0.        , -0.5       ,  0.        ],
       [ 0.08333333,  0.125     ,  0.        ],
       [ 0.08333333,  0.20833333,  0.        ],
       [ 0.08333333,  0.29166667,  0.        ],
       [ 0.08333333,  0.375     ,  0.        ],
       [ 0.08333333,  0.45833333,  0.        ],
       [ 0.16666667,  0.25      ,  0.        ],
       [ 0.16666667,  0.33333333,  0.        ],
       [ 0.16666667,  0.41666667,  0.        ],
       [ 0.16666667,  0.5       ,  0.        ],
       [ 0.25      ,  0.375     ,  0.        ],
       [ 0.25      ,  0.45833333,  0.        ],
       [ 0.33333333,  0.5       ,  0.        ]])

In [51]:
dft.lattice/7.643

array([[ 1.       ,  0.       ,  0.       ],
       [-0.5      ,  0.8660254,  0.       ],
       [ 0.       ,  0.       ,  8.       ]])

In [48]:
dft.alat

7.643

In [11]:
inp = I.YamboInput(args='yambo -x -V rl',folder=run_dir)
inp

{'args': 'yambo -x -V rl',
 'folder': 'GW_analysis',
 'filename': 'yambo.in',
 'arguments': ['HF_and_locXC'],
 'variables': {'FFTGvecs': [52407.0, 'RL'],
  'SE_Threads': [0.0, ''],
  'EXXRLvcs': [356277.0, 'RL'],
  'VXCRLvcs': [356277.0, 'RL'],
  'QPkrange': [[1, 19, 1, 60], '']}}